In [0]:

'''
AiCore Pinterest Data Pipeline Project
Load Pinterest data from S3; clean and analyse the data with PySpark.
This code is intended to run in a Databricks notebook.
Author: Kristina Gorkovskaya
Date: 2023-09-30
'''

import pandas as pd
import pyspark
import pyspark.sql.functions as F
from pyspark.sql.types import ArrayType, DoubleType, IntegerType, StringType

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 100)

user_id = '0ec858bf1407'
mount_name = 'pinterest'

def load_data(topic_suffix: str, user_id: str = user_id, mount_name: str = mount_name) -> pyspark.sql.dataframe.DataFrame:
  '''Load JSON data from S3 into a PySpark DataFrame; print the schema to view column names and data types'''
  print(f'Loading topic {user_id}.{topic_suffix}...', end=' ')
  file_location = f'/mnt/{mount_name}/topics/{user_id}.{topic_suffix}/partition=0/*.json'
  file_type = 'json'
  df = spark.read.format(file_type).option('inferSchema', True).load(file_location)
  
  print(f'{df.count():,} records loaded.')
  df.printSchema()
  return df

def show_non_numeric_patterns(df: pyspark.sql.dataframe.DataFrame, col: str) -> pyspark.sql.dataframe.DataFrame:
  '''Look for non-numeric patterns in a field that is expected to be numeric.'''
  new_col = col + '_pattern'
  df = df.withColumn(new_col, F.regexp_replace(col, '[0-9]+', '9'))
  df.groupBy(new_col).count().show()
  return df


--------------------------------------------------------------------------- 
 NameError Traceback (most recent call last)
 <command-319053041177791> in <module> 
 ----> 1 df = df . withColumn ( new_col , F . regexp_replace ( col , '[0-9]+' , '9' ) ) 

 NameError : name 'new_col' is not defined

In [0]:
#############################################################################################################
# TASK 1: Clean the df_pin DataFrame
#############################################################################################################
# Load data from the pins topic into df_pin.
df_pin = load_data('pin')

Loading topic 0ec858bf1407.pin... 1,962 records loaded.
root
-- category: string (nullable = true)
-- description: string (nullable = true)
-- downloaded: long (nullable = true)
-- follower_count: string (nullable = true)
-- image_src: string (nullable = true)
-- index: long (nullable = true)
-- is_image_or_video: string (nullable = true)
-- poster_name: string (nullable = true)
-- save_location: string (nullable = true)
-- tag_list: string (nullable = true)
-- title: string (nullable = true)
-- unique_id: string (nullable = true)

In [0]:
# (1) Replace empty entries and entries with no relevant data in each column with Nones
# Start by counting nulls
nulls = {col: df_pin.filter((df_pin[col].isNull()) | F.isnan(col)).count() for col in df_pin.columns}
print(nulls)

# Replace nans with None
df_pin = df_pin.replace(float('nan'), None)

# Replace empty strings with None
df_pin = df_pin.select([F.when(df_pin[c] == "", None).otherwise(df_pin[c]).alias(c) for c in df.columns])

# Count nulls again
nulls = {col: df_pin.filter((df_pin[col].isNull()) | F.isnan(col)).count() for col in df_pin.columns}
print(nulls)

{'category': 0, 'description': 0, 'downloaded': 0, 'follower_count': 0, 'image_src': 0, 'index': 0, 'is_image_or_video': 0, 'poster_name': 0, 'save_location': 0, 'tag_list': 0, 'title': 0, 'unique_id': 0}
{'category': 0, 'description': 0, 'downloaded': 0, 'follower_count': 0, 'image_src': 0, 'index': 0, 'is_image_or_video': 0, 'poster_name': 0, 'save_location': 0, 'tag_list': 0, 'title': 1, 'unique_id': 0}

In [0]:
# (2) Perform the necessary transformations on the follower_count to ensure every entry is a number. 
# First look for patterns in the data - what non-numeric characters are present?
df_pin = show_non_numeric_patterns(df_pin, 'follower_count')

+----------------------+-----+
follower_count_pattern|count|
+----------------------+-----+
 9k| 1454|
 9| 342|
 9M| 136|
 User Info Error| 30|
+----------------------+-----+

In [0]:
# Create a multiplier column
# Assumptions: a suffix of "k" in follower_count means a multiplier of 1,000; a suffix of "M" means 10^6
df_pin = df_pin.withColumn('follower_count_multiplier', 
                           F.when(df_pin.follower_count.like('%k'), 1000).
                           when(df_pin.follower_count.like('%M'), 1000000).
                           otherwise(1))

# Check that multiplier has been created correctly
df_pin.groupBy('follower_count_pattern', 'follower_count_multiplier').count().show()

+----------------------+-------------------------+-----+
follower_count_pattern|follower_count_multiplier|count|
+----------------------+-------------------------+-----+
 9k| 1000| 1454|
 9M| 1000000| 136|
 9| 1| 342|
 User Info Error| 1| 30|
+----------------------+-------------------------+-----+

In [0]:
# Parse numeric data from follower_count and apply multiplier to the parsed values
df_pin = df_pin.withColumn('follower_count_numeric',
                           F.when(df_pin.follower_count == 'User Info Error', None).
                           otherwise(F.regexp_replace('follower_count', '[^0-9]+', '')).
                           cast(IntegerType()) * df_pin.follower_count_multiplier)

# Show a random sample of 5 rows rows for each pattern to make sure the calculations are correct
df_pin.createOrReplaceTempView('df_pin')
sql = """
select distinct 
    follower_count, 
    follower_count_numeric 
from (
    select 
        follower_count, 
        follower_count_numeric, 
        row_number() over (partition by follower_count_pattern order by random()) as rn
    from df_pin 
)
where rn <= 5
order by follower_count_numeric
"""
spark.sql(sql).show(truncate=False)

+---------------+----------------------+
follower_count |follower_count_numeric|
+---------------+----------------------+
User Info Error|null |
0 |0 |
5 |5 |
18 |18 |
221 |221 |
263 |263 |
1k |1000 |
4k |4000 |
5k |5000 |
6k |6000 |
20k |20000 |
1M |1000000 |
2M |2000000 |
5M |5000000 |
8M |8000000 |
+---------------+----------------------+

In [0]:
# Remove intermediate columns
df_pin = df_pin.withColumn('follower_count', df_pin.follower_count_numeric)
df_pin = df_pin.drop('follower_count_numeric', 'follower_count_pattern', 'follower_count_multiplier')
df_pin.printSchema()

root
-- category: string (nullable = true)
-- description: string (nullable = true)
-- downloaded: long (nullable = true)
-- follower_count: integer (nullable = true)
-- image_src: string (nullable = true)
-- index: long (nullable = true)
-- is_image_or_video: string (nullable = true)
-- poster_name: string (nullable = true)
-- save_location: string (nullable = true)
-- tag_list: string (nullable = true)
-- title: string (nullable = true)
-- unique_id: string (nullable = true)

In [0]:
# (3) Ensure that each column containing numeric data has a numeric data type
# Start by displaying the first 10 rows to identify candidate columns
df_pin.show(10, truncate=True)

+--------------+--------------------+----------+--------------+--------------------+-----+-----------------+--------------------+--------------------+--------------------+---------------------+--------------------+
 category| description|downloaded|follower_count| image_src|index|is_image_or_video| poster_name| save_location| tag_list| title| unique_id|
+--------------+--------------------+----------+--------------+--------------------+-----+-----------------+--------------------+--------------------+--------------------+---------------------+--------------------+
event-planning|Το όνομα που επέλ...| 1| 4|https://i.pinimg....| 4387| image|Manosbojana Katsa...|Local save in /da...|Diy Flowers,Flowe...| Βάπτιση: H παραμυ...|ae5e7377-f1bd-4ac...|
 home-decor|Традиционные швед...| 1| 136000|https://i.pinimg....| 6717| image|PUFIK Interiors &...|Local save in /da...|Scandinavian Cott...|〚 Уютные шведские...|bc5ab9ee-505e-44f...|
 home-decor|Традиционные швед...| 1| 136000|https://i.pinimg....| 6717| image|PUFIK Interiors &...|Local save in /da...|Scandinavian Cott...|〚 Уютные шведские...|bc5ab9ee-505e-44f...|
event-planning|15.1k Likes, 83 C...| 1| 311|https://i.pinimg....| 4858| image| Marie Bradford|Local save in /da...|Diy Birthday Deco...| THE EVENT COLLECT...|58101415-9273-431...|
event-planning|Wow your guests! ...| 1| 1000|https://i.pinimg....| 4608| image|Iconica Design | ...|Local save in /da...|Christmas Party B...| Virtual Baby Show...|d234e56f-5b18-4ef...|
 home-decor|6,636 Likes, 141 ...| 1| 394|https://i.pinimg....| 6633| image| Sarah Martin|Local save in /da...|Diy Kitchen Cabin...| The Cottage Journ...|d136f6bc-840d-44f...|
 christmas|Features: Materia...| 1| 5000|https://i.pinimg....| 1706| image| Wear24-7|Local save in /da...|Merry Christmas T...| Standing Figurine...|b5c8a1b5-9e90-452...|
 christmas|Features: Materia...| 1| 5000|https://i.pinimg....| 1706| image| Wear24-7|Local save in /da...|Merry Christmas T...| Standing Figurine...|b5c8a1b5-9e90-452...|
 christmas|Features: Materia...| 1| 5000|https://i.pinimg....| 1706| image| Wear24-7|Local save in /da...|Merry Christmas T...| Standing Figurine...|b5c8a1b5-9e90-452...|
 christmas|❤️ ❤️ MERRY CHRIS...| 1| 784|https://i.pinimg....| 1676| image| paupoo|Local save in /da...|Christmas Hanging...| PAUPOO™ Pre-lit A...|3ed92c2d-9cca-4cc...|
+--------------+--------------------+----------+--------------+--------------------+-----+-----------------+--------------------+--------------------+--------------------+---------------------+--------------------+
only showing top 10 rows

In [0]:
# It looks like the following columns are numeric: downloaded, index, follower_count
# follower_count has already been transformed; let's deal with the remaining columns
numeric_cols = ['downloaded', 'index']
for col in numeric_cols:
    df_pin = show_non_numeric_patterns(df_pin, col) 

+------------------+-----+
downloaded_pattern|count|
+------------------+-----+
 9| 1962|
+------------------+-----+

+-------------+-----+
index_pattern|count|
+-------------+-----+
 9| 1962|
+-------------+-----+

In [0]:
# The downloaded and index columns are already populated with integers only; therefore 
# no transformations are required before casting data as int
for col in numeric_cols:
    df_pin = df_pin.withColumn(col, df_pin[col].cast(IntegerType()))

# Drop intermediate columns created by the show_non_numeric_patterns function
df_pin = df_pin.drop(*[c + '_pattern' for c in numeric_cols])

# Print schema to make sure the transformations were performed as expected
df_pin.printSchema()

root
-- category: string (nullable = true)
-- description: string (nullable = true)
-- downloaded: integer (nullable = true)
-- follower_count: integer (nullable = true)
-- image_src: string (nullable = true)
-- index: integer (nullable = true)
-- is_image_or_video: string (nullable = true)
-- poster_name: string (nullable = true)
-- save_location: string (nullable = true)
-- tag_list: string (nullable = true)
-- title: string (nullable = true)
-- unique_id: string (nullable = true)

In [0]:
# (4) Clean the data in the save_location column to include only the save location path
# Start by manually reviewing the data to look for patterns.
# Display unique values (cache the DataFrame to speed up any subsequent calls to distinct())
num_unique_vals = df_pin.select('save_location').distinct().count()
df_pin.cache()

print(f'{num_unique_vals:,} unique values')
if num_unique_vals < 50:
    df_pin.select('save_location').distinct().show(truncate=False)
else:
    df_pin.select('save_location').show(50, truncate=False)

13 unique values
+----------------------------------+
save_location |
+----------------------------------+
Local save in /data/vehicles |
Local save in /data/diy-and-crafts|
Local save in /data/event-planning|
Local save in /data/art |
Local save in /data/tattoos |
Local save in /data/home-decor |
Local save in /data/christmas |
Local save in /data/education |
Local save in /data/finance |
Local save in /data/mens-fashion |
Local save in /data/travel |
Local save in /data/quotes |
Local save in /data/beauty |
+----------------------------------+

In [0]:
# All the records follow a consistent pattern; can therefore be cleaned using regexp_replace
df_pin = df_pin.withColumn('save_location', F.regexp_replace('save_location', 'Local save in ', ''))
df_pin.select('save_location').distinct().show(truncate=False)

+--------------------+
save_location |
+--------------------+
/data/home-decor |
/data/diy-and-crafts|
/data/christmas |
/data/tattoos |
/data/art |
/data/event-planning|
/data/vehicles |
/data/education |
/data/travel |
/data/finance |
/data/mens-fashion |
/data/quotes |
/data/beauty |
+--------------------+

In [0]:
# (6) Rename the index column to ind, and reorder the columns.
cols = [    
    'ind',
    'unique_id',
    'title',
    'description',
    'follower_count',
    'poster_name',
    'tag_list',
    'is_image_or_video',
    'image_src',
    'save_location',
    'category'
    ]
df_pin = df_pin.withColumnRenamed('index', 'ind').select(cols)
df_pin.printSchema()

root
-- ind: integer (nullable = true)
-- unique_id: string (nullable = true)
-- title: string (nullable = true)
-- description: string (nullable = true)
-- follower_count: integer (nullable = true)
-- poster_name: string (nullable = true)
-- tag_list: string (nullable = true)
-- is_image_or_video: string (nullable = true)
-- image_src: string (nullable = true)
-- save_location: string (nullable = true)
-- category: string (nullable = true)

In [0]:
#############################################################################################################
# TASK 2: Clean the df_geo (geolocation) DataFrame
#############################################################################################################
# Load data from the geo topic into df_geo.
df_geo = load_data('geo')

Loading topic 0ec858bf1407.geo... 1,904 records loaded.
root
-- country: string (nullable = true)
-- ind: long (nullable = true)
-- latitude: double (nullable = true)
-- longitude: double (nullable = true)
-- timestamp: string (nullable = true)

In [0]:
# (1) Create a new column coordinates that contains an array based on the latitude and longitude columns
# Start by defining a UDF that takes multiple columns and returns an array.
def make_list(*args):
    return list(args)

udf_make_list = F.udf(make_list, ArrayType(DoubleType()))

# Then apply the UDF to latitude and longitude
df_geo = df_geo.withColumn('coordinates', udf_make_list('latitude', 'longitude'))
df_geo.select('latitude', 'longitude', 'coordinates').show(10, truncate=False)

# (2) Drop the latitude and longitude columns from the DataFrame
df_geo = df_geo.drop('latitude', 'longitude')
df_geo.printSchema()

+--------+---------+--------------------+
latitude|longitude|coordinates |
+--------+---------+--------------------+
-82.9272|-150.346 |[-82.9272, -150.346]|
-86.5675|-149.565 |[-86.5675, -149.565]|
-83.7472|8.65953 |[-83.7472, 8.65953] |
-87.013 |133.062 |[-87.013, 133.062] |
-32.8885|-170.295 |[-32.8885, -170.295]|
-32.8885|-170.295 |[-32.8885, -170.295]|
-88.4642|-171.061 |[-88.4642, -171.061]|
-88.4642|-171.061 |[-88.4642, -171.061]|
-71.6607|-149.206 |[-71.6607, -149.206]|
-77.9931|-175.682 |[-77.9931, -175.682]|
+--------+---------+--------------------+
only showing top 10 rows

root
-- country: string (nullable = true)
-- ind: long (nullable = true)
-- timestamp: string (nullable = true)
-- coordinates: array (nullable = true)
 |-- element: double (containsNull = true)

In [0]:
# (3) Convert the timestamp column from a string to a timestamp data type
# Start by looking for patterns in the timestamp field
df_geo.groupBy(F.regexp_replace('timestamp', '[0-9]', '9').alias('timestamp_pattern')).count().show()

+-------------------+-----+
 pattern|count|
+-------------------+-----+
9999-99-99 99:99:99| 1904|
+-------------------+-----+

In [0]:
# All timestamps are in a consistent format. Convert using sql.functions.to_timestamp;
# Display records to verify that timestamps were parsed correctly
df_geo = df_geo.withColumn('timestamp_converted', F.to_timestamp('timestamp'))
df_geo.select('timestamp', 'timestamp_converted').show(10)

+-------------------+-------------------+
 timestamp|timestamp_converted|
+-------------------+-------------------+
2022-03-15 01:46:32|2022-03-15 01:46:32|
2022-09-02 11:34:28|2022-09-02 11:34:28|
2021-04-01 00:56:57|2021-04-01 00:56:57|
2020-01-09 19:18:54|2020-01-09 19:18:54|
2021-06-29 19:56:04|2021-06-29 19:56:04|
2021-06-29 19:56:04|2021-06-29 19:56:04|
2022-05-27 11:30:59|2022-05-27 11:30:59|
2022-05-27 11:30:59|2022-05-27 11:30:59|
2019-09-27 19:06:43|2019-09-27 19:06:43|
2022-03-19 17:29:42|2022-03-19 17:29:42|
+-------------------+-------------------+
only showing top 10 rows

In [0]:
# Replace the timestamp column with converted values and drop the intermediate column
df_geo = df_geo.withColumn('timestamp', df_geo.timestamp_converted).drop('timestamp_converted')

In [0]:
# (4) Reorder columns
df_geo = df_geo.select('ind', 'country', 'coordinates', 'timestamp')
df_geo.printSchema()

root
-- ind: long (nullable = true)
-- country: string (nullable = true)
-- coordinates: array (nullable = true)
 |-- element: double (containsNull = true)
-- timestamp: timestamp (nullable = true)